In [1]:
import random
import time
import pandas as pd
import requests
from scrapy.http import TextResponse
from fake_useragent import UserAgent

In [5]:
# 크롤링한 맛집 데이터 로드
df = pd.read_excel("./nav_df.xlsx")
df.head()

,id,dong,name,tel,addr,bizhour,category,context,menu,reviewcount,microreview,thumbnail,lng,lat,rating
0,37450428,사직동,박순례 손말이고기 산정집 광화문점,02-310-9636,서울특별시 종로구 당주동 3-2 선덕빌딩 2층,일요일 휴무 명절연휴 휴무 | 매일 11:30~14:00 점심 | 매일 14:00~...,"['한식', '소고기구이']","['시래기', '고기말이', '한우', '고급스러운', '나들이']","한우말이고기 21,000 | 한우내장볶음 21,000 | 한우내장전골 25,000 ...",203,[],https://ldb-phinf.pstatic.net/20200930_73/1601...,126.974200,37.572380,NaN
1,1934318307,사직동,한옥 달,010-8706-3392,서울특별시 종로구 체부동 68,평일 11:30~22:00 (L.O. 20:30) | 평일 15:30~17:00 B...,"['이탈리아음식', '스파게티,파스타전문']",[],"단호박 크림 스프 15,000 | 아보카도 허머스 16,000 | 리코타 치즈 샐러...",322,[],https://ldb-phinf.pstatic.net/20190808_53/1565...,126.971983,37.576946,NaN
2,19489992,사직동,할매집,02-735-2608,서울특별시 종로구 내자동 108,매일 12:00~21:30,"['한식', '족발,보쌈']","['감자탕', '족발', '미슐랭가이드', '우아한', '콩국수']","족발 30,000 | 감자탕(중) 25,000 | 감자탕(대) 30,000",390,"['콩나물 넣은 감자탕', '구수하고 푸짐한 감자탕을 먹을 수 있는 식신로드 맛집'...",http://blogfiles.naver.net/20150910_18/xxxyeji...,126.971475,37.575660,NaN
3,1010940645,사직동,디퍼카페테리아,010-9160-4482,서울특별시 종로구 체부동 2-2,주말 12:00~19:30 | 수요일 12:00~19:30 | 목요일 12:00~1...,"['음식점', '카페,디저트']",[],"아메리카노 3,500 | 카페라떼 4,000 | 그라니따 5,500 | 스폴리아텔라...",39,[],https://ldb-phinf.pstatic.net/20200908_251/159...,126.970393,37.579038,NaN
4,676217924,사직동,후라토식당 경복궁 본점,02-730-7090,서울특별시 종로구 당주동 5,매일 11:30~22:00 평일 브레이크타임 15:00-17:00,"['일식', '일식당']","['오므라이스', '데이트', '일본가정식', '프라이빗', '돈가스']","규카츠 15,000 | 반숙 오므라이스 11,000 | 야마가타 민치카레 12,00...",497,[],https://ldb-phinf.pstatic.net/20200804_260/159...,126.974311,37.572032,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21200 entries, 0 to 21199
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           21200 non-null  int64  
 1   dong         21200 non-null  object 
 2   name         21200 non-null  object 
 3   tel          21013 non-null  object 
 4   addr         21200 non-null  object 
 5   bizhour      20612 non-null  object 
 6   category     21198 non-null  object 
 7   context      21200 non-null  object 
 8   menu         21036 non-null  object 
 9   reviewcount  21200 non-null  int64  
 10  microreview  21200 non-null  object 
 11  thumbnail    20877 non-null  object 
 12  lng          21200 non-null  float64
 13  lat          21200 non-null  float64
 14  rating       0 non-null      float64
dtypes: float64(3), int64(2), object(10)
memory usage: 2.4+ MB


In [ ]:
# 네이버 지도에서 크롤링한 id값으로 네이버 플레이스에서 평점 크롤링
params = {'cookie' : 'NNB=LUURYNMJ4C6F6; nx_ssl=2; NRTK=ag#30s_gr#4_ma#2_si#2_en#2_sp#2; page_uid=UJ0nKsp0YidssjcF2NRssssssf0-196483; BMR=s=1606280225697&r=https%3A%2F%2Fm.map.naver.com%2Fsearch2%2Fsearch.nhn%3Fquery%3D%25EC%2582%25AC%25EC%25A7%2581%25EB%258F%2599%2520%25EB%25A7%259B%25EC%25A7%2591%26sm%3Dhty%26style%3Dv5&r2=https%3A%2F%2Fm.map.naver.com%2Fsearch2%2Fsearch.nhn%3Fquery%3D%25EC%2582%25AC%25EC%25A7%2581%25EB%258F%2599%2520%25EB%25A7%259B%25EC%25A7%2591%26sm%3Dhty%26style%3Dv5; _naver_usersession_=lnInDLBwlOnd3hhzdqRK2Geb',
         'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'}
for i in range(0, len(df)):
    url = "https://m.place.naver.com/restaurant/{}/home".format(df["id"][i])
    time.sleep(5)
    try:
        response = requests.get(url, params=params)
        print(df["name"][i], response)
        dom = TextResponse(response.url, body=response.text, encoding='utf-8')
        df["rating"][i] = dom.xpath('//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]/div/span[1]/em/text()')[0].extract()
    except:
        print(df["name"][i], response)
        df["rating"][i] = False
        time.sleep(600)

박순례 손말이고기 산정집 광화문점 <Response [200]>


<ipython-input-9-ca4a5e7849bd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rating"][i] = dom.xpath('//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]/div/span[1]/em/text()')[0].extract()


한옥 달 <Response [200]>
할매집 <Response [200]>
디퍼카페테리아 <Response [200]>
후라토식당 경복궁 본점 <Response [200]>
도토리브라더스 <Response [200]>
대접 광화문본점 <Response [200]>
심퍼티쿠시 경복궁점 <Response [200]>
밍고네식탁 <Response [200]>
가스트로통 <Response [200]>
오가와 <Response [200]>
몽쉐프 <Response [200]>
서촌계단집 <Response [200]>
암소서울 <Response [200]>
알레그리아 덕수궁 디팰리스점 <Response [200]>
티엔미미 <Response [200]>
체부동잔치집 <Response [200]>
카페아르크 <Response [200]>
올리바가든 <Response [200]>
올리바가든 <Response [200]>


<ipython-input-9-ca4a5e7849bd>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rating"][i] = False
/Users/jaecheolchoi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


사발 <Response [200]>
고가빈커리하우스 <Response [200]>
고가빈커리하우스 <Response [200]>
네기 실비 <Response [200]>
네기 실비 <Response [200]>


#### 망고플레이트(5), 다이닝코드(100), 네이버리뷰 수, 블루리본 수, 메뉴판닷컴(5)
- 다이닝코드 점수.MinMaxScale() * 2
- 망고플레이트 점수.MinMaxScale() * 2
- 메뉴판닷컴 점수.MinMaxScale() * 2
- (블루리본수+1).MinMaxScale() * 3
- 네이버리뷰수.MinMaxScale() * 1

    - 총 10점 만점
    - 해당 맛집 데이터가 없는 플랫폼에서는 해당 플랫폼 값의 mean값 적용